In [1]:
import torch
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

# Load the pre-trained DeepLabV3+ model
model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet50', pretrained=True)
model.eval()

# Load and preprocess the input image
image_path = 'path_to_image.jpg'  # Replace with the path to your image
image = Image.open(image_path).convert('RGB')
preprocess = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(image).unsqueeze(0)

# Make predictions
with torch.no_grad():
    output = model(input_tensor)['out']
predictions = output.argmax(1).squeeze().cpu().numpy()

# Visualize the segmentation mask
label_colors = np.array([(0, 0, 0),  # Background
                        (128, 0, 0),  # Class 1
                        (0, 128, 0),  # Class 2
                        (0, 0, 128)])  # Class 3
colored_mask = label_colors[predictions]
segmentation_mask = Image.fromarray(colored_mask.astype(np.uint8))

# Display the original image and segmentation mask
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(segmentation_mask)
plt.title('Segmentation Mask')
plt.axis('off')

plt.show()


Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /Users/christianbasile/.cache/torch/hub/v0.10.0.zip
/Users/christianbasile/opt/anaconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/christianbasile/opt/anaconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/deeplabv3_resnet50_coco-cd0a2569.pth" to /Users/christianbasile/.cache/torch/hub/checkpoints/deeplabv3_resnet50_coco-cd0a2569.p

  0%|          | 0.00/161M [00:00<?, ?B/s]

FileNotFoundError: [Errno 2] No such file or directory: 'path_to_image.jpg'

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.segmentation import mean_shift

# Generate a random image
image_shape = (512, 512, 3)  # Image dimensions (height, width, channels)
image = np.random.randint(0, 256, image_shape, dtype=np.uint8)

# Perform MeanShift segmentation
segmented_image = mean_shift(image, bandwidth=40)

# Display the input image and segmented image
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.imshow(image)
ax1.set_title('Input Image')
ax1.axis('off')
ax2.imshow(segmented_image)
ax2.set_title('Segmented Image')
ax2.axis('off')
plt.show()


ImportError: cannot import name 'mean_shift' from 'skimage.segmentation' (/Users/christianbasile/opt/anaconda3/lib/python3.8/site-packages/skimage/segmentation/__init__.py)

In [ ]:
import cv2
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth

# Load the video
video_path = 'path_to_video.mp4'  # Replace with the path to your video file
cap = cv2.VideoCapture(video_path)

# Initialize the MeanShift algorithm
ms = MeanShift()

# Read the first frame from the video
ret, frame = cap.read()

# Perform MeanShift segmentation on the first frame
roi = frame[100:400, 100:400]  # Region of interest for segmentation
roi_hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
flat_roi = roi_hsv.reshape((-1, 3))
bandwidth = estimate_bandwidth(flat_roi, quantile=0.2, n_samples=500)
ms.set_bandwidth(bandwidth)
ms.fit(flat_roi)

# Continuously process video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Perform MeanShift segmentation on the frame
    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    flat_frame = frame_hsv.reshape((-1, 3))
    labels = ms.predict(flat_frame)
    segmented_frame = labels.reshape(frame_hsv.shape[:2])
    
    # Display the segmented frame
    cv2.imshow('Segmented Frame', segmented_frame)
    
    # Detect movement in the x and y directions
    flow = cv2.calcOpticalFlowFarneback(prev_frame, frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    flow_x = np.mean(flow[..., 0])
    flow_y = np.mean(flow[..., 1])
    
    # Print the movement in the x and y directions
    print('Movement (x, y):', flow_x, flow_y)
    
    # Store the current frame as the previous frame for the next iteration
    prev_frame = frame.copy()
    
    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()


In [8]:
import os

os.chdir('/Users/christianbasile/Downloads')

In [9]:
import cv2
import pytesseract

# Load the image
image_path = 'images.jpeg'  # Replace with the path to your image
image = cv2.imread(image_path)

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply image preprocessing (if needed)
# For example, you can perform thresholding, noise removal, or other techniques
# gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

# Perform OCR to extract text from the image
extracted_text = pytesseract.image_to_string(gray)

# Organize the extracted text
lines = extracted_text.split('\n')
lines = [line.strip() for line in lines if line.strip()]

# Print the organized text
for line in lines:
    print(line)


[ WARN:0@481.560] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('images.jpeg'): can't open/read file: check file path/integrity


error: OpenCV(4.6.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
